Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/radiomics_example 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/radiomics_example


Import all the lib

In [2]:
try:
  from radiomics import featureextractor
except ImportError as e:
  !pip install pyradiomics
  from radiomics import featureextractor

try:
  import nrrd
except ImportError as e:
  !pip install pynrrd
  import nrrd

import six
import os
import numpy as np
import nibabel as nib
from ipywidgets import *
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

Assign the file path

In [3]:
mask_name   = './crc-segmentation-0.nii'
source_name = './crc-volume-0.nii'

mask = nib.load(mask_name)
mask_array = np.array(mask.dataobj).reshape(mask.shape).transpose(1, 0, 2)

source = nib.load(source_name)
source_array = np.array(source.dataobj).reshape(source.shape).transpose(1, 0, 2)

In [4]:
(dim_x, dim_y, dim_z) = (source.header['dim'][1:4])
(size_x, size_y, size_z) = (source.header['pixdim'][1:4])
source_max, source_min = np.max(source_array[:,:,:]), np.min(source_array[:,:,:])
mask_max, mask_min = np.max(mask_array[:,:,:]), np.min(mask_array[:,:,:])

This widget only used for checking data

In [5]:
def update(slice):    
  src_img = source_array[:,:,slice]
  msk_img = mask_array[:,:,slice]
  fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))
  ax[0].imshow(src_img, cmap=plt.cm.gray, vmax=source_max, vmin=source_min)
  ax[1].imshow(msk_img, cmap=plt.cm.hot, vmax=mask_max, vmin=mask_min)
  ax[2].imshow(src_img, cmap=plt.cm.gray, vmax=source_max, vmin=source_min)
  ax[2].imshow(msk_img, cmap=plt.cm.hot, vmax=mask_max, vmin=mask_min, alpha = 0.3)

interact(update, slice = widgets.IntSlider(value=dim_z//2, min=0, max=dim_z-1))

interactive(children=(IntSlider(value=497, description='slice', max=993), Output()), _dom_classes=('widget-int…

<function __main__.update>

In [6]:
from skimage.measure import label, regionprops

def extract_bboxes(mask):
  results = []

  label_mask = label(mask)
  for region in regionprops(label_mask):
    x,y,z = np.round(region.centroid).astype(np.int)
    gray = mask[x,y,z]
    label_num = region.label
    minx, miny, minz, maxx, maxy, maxz = region.bbox

    values = (gray, label_num, minx, miny, minz, maxx, maxy, maxz)
    results.append(values)
  
  return results

def crop_volume(source, mask, info):
  (gray, label_num, minx, miny, minz, maxx, maxy, maxz) = info
  msk = mask[minx-1:maxx+1, miny-1:maxy+1, minz-1:maxz+1]
  src = source[minx-1:maxx+1, miny-1:maxy+1, minz-1:maxz+1]
  # msk[msk != gray] = 0
  src[msk != gray] = -1000

  return src, msk


Get the bounding boxes in 3-D space

In [7]:
bbox_results = extract_bboxes(mask_array)
print(bbox_results)

[(1, 1, 174, 45, 396, 404, 305, 633), (2, 2, 207, 272, 450, 219, 282, 453), (2, 3, 210, 272, 454, 221, 282, 457), (2, 4, 210, 276, 445, 219, 285, 448), (2, 5, 211, 279, 438, 224, 288, 439), (2, 6, 213, 277, 432, 230, 288, 435), (2, 7, 294, 84, 408, 305, 95, 414), (2, 8, 327, 119, 418, 346, 139, 433)]


Export the cropped results and execute PyRadiomics

In [8]:
radiomics_results = []
extractor = featureextractor.RadiomicsFeatureExtractor()

for idx in range(0, len(bbox_results)):  
  # print(bbox_results[idx])
  subRegion, msk = crop_volume(source_array, mask_array, bbox_results[idx])

  # Save NRRD
  img_path_to = './src.nrrd'
  msk_path_to = './msk.nrrd'
  nrrd.write(img_path_to, subRegion)
  nrrd.write(msk_path_to, msk)
 

  radiomics_result = extractor.execute(img_path_to, msk_path_to, voxelBased=False) #maybe you can change the voxelBased to True
  radiomics_results.append(radiomics_result)

  os.remove(img_path_to)
  os.remove(msk_path_to)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Export the results

In [9]:
for idx in range(len(radiomics_results)):
  radiomics_result = radiomics_results[idx]

  for key, val in six.iteritems(radiomics_result):
    print("\t%s: %s" %(key, val))

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.19.5
	diagnostics_Versions_SimpleITK: 2.0.2
	diagnostics_Versions_PyWavelet: 1.1.1
	diagnostics_Versions_Python: 3.7.10
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
	diagnostics_Image-original_Hash: cba83d50f3e5010eb13ca53b223c333b8b19f861
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (1.0, 1.0, 1.0)
	diagnostics_Image-original_Size: (232, 262, 239)
	diagnostics_Image-original_Mean: -663.1734090175679
	diagnostics_Image-original_Minimum: -1000.0
	diagnostics_Image-original_Maximum: 2981.0
	diagn

In [10]:
def update2(slice):    
  src_img = subRegion[:,:,slice]  
  msk_img = msk[:,:,slice]
  fig2, ax2 = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
  ax2[0].imshow(src_img, cmap=plt.cm.gray, vmax=source_max, vmin=source_min)
  ax2[1].imshow(msk_img, cmap=plt.cm.hot, vmax=mask_max, vmin=mask_min)


interact(update2, slice = widgets.IntSlider(value=np.shape(subRegion)[2]//2, min=0, max=np.shape(subRegion)[2]-1))

interactive(children=(IntSlider(value=8, description='slice', max=16), Output()), _dom_classes=('widget-intera…

<function __main__.update2>